In [17]:
import struct

In [18]:
# Constantes SHA-256
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

In [19]:
# Funciones auxiliares
def rotr(x, n, w=32):
    return (x >> n) | ((x << (w - n)) & (2**w - 1))

def sha256_compress(chunk, H):
    # Extender los bloques
    W = list(struct.unpack(">16L", chunk)) + [0] * 48
    for t in range(16, 64):
        s0 = rotr(W[t - 15], 7) ^ rotr(W[t - 15], 18) ^ (W[t - 15] >> 3)
        s1 = rotr(W[t - 2], 17) ^ rotr(W[t - 2], 19) ^ (W[t - 2] >> 10)
        W[t] = (W[t - 16] + s0 + W[t - 7] + s1) & 0xFFFFFFFF

    # Inicializar variables
    a, b, c, d, e, f, g, h = H

    # Rondas de compresión
    for t in range(64):
        S1 = rotr(e, 6) ^ rotr(e, 11) ^ rotr(e, 25)
        ch = (e & f) ^ (~e & g)
        temp1 = (h + S1 + ch + K[t] + W[t]) & 0xFFFFFFFF
        S0 = rotr(a, 2) ^ rotr(a, 13) ^ rotr(a, 22)
        maj = (a & b) ^ (a & c) ^ (b & c)
        temp2 = (S0 + maj) & 0xFFFFFFFF
        h = g
        g = f
        f = e
        e = (d + temp1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (temp1 + temp2) & 0xFFFFFFFF

    # Actualizar valores de hash
    return [
        (H[0] + a) & 0xFFFFFFFF,
        (H[1] + b) & 0xFFFFFFFF,
        (H[2] + c) & 0xFFFFFFFF,
        (H[3] + d) & 0xFFFFFFFF,
        (H[4] + e) & 0xFFFFFFFF,
        (H[5] + f) & 0xFFFFFFFF,
        (H[6] + g) & 0xFFFFFFFF,
        (H[7] + h) & 0xFFFFFFFF,
    ]

In [20]:
# Preprocesamiento
def sha256_padding(message):
    message_bytes = bytearray(message, 'ascii')
    original_length = len(message_bytes) * 8
    message_bytes.append(0x80)
    while len(message_bytes) % 64 != 56:
        message_bytes.append(0)
    message_bytes += struct.pack(">Q", original_length)
    return message_bytes



In [21]:
# Algoritmo SHA-256
def sha256(message):
    H = [
        0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
        0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19,
    ]
    padded_message = sha256_padding(message)
    for i in range(0, len(padded_message), 64):
        chunk = padded_message[i:i + 64]
        H = sha256_compress(chunk, H)
    return ''.join(f'{x:08x}' for x in H)

# Prueba del algoritmo
mensaje = "abc"
print("Hash SHA-256:", sha256(mensaje))

Hash SHA-256: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad


# Verificar Algoritmo

In [22]:
import hashlib
from sha256 import sha256

In [23]:

def calculate_sha256_hash(message):
    """
    Calcula el hash SHA-256 de un mensaje utilizando la librería hashlib.
    """
    # Convertir el mensaje a bytes
    message_bytes = message.encode('utf-8')
    # Crear el objeto hash
    sha256_hash = hashlib.sha256()
    # Alimentar el mensaje al objeto hash
    sha256_hash.update(message_bytes)
    # Obtener el hash final en formato hexadecimal
    return sha256_hash.hexdigest()

In [24]:
# Mensaje de ejemplo
mensaje = "abc"

# Calcular hash con hashlib
hash_resultado = calculate_sha256_hash(mensaje)

# Imprimir resultado
print(f"Mensaje: {mensaje}")
print(f"Hash SHA-256: {hash_resultado}")

# Hash esperado para "abc" (verificado en estándares y herramientas externas)
hash_esperado = sha256(mensaje)
print(f"Hash esperado: {hash_esperado}")
print(f"¿El hash coincide?: {'Sí' if hash_resultado == hash_esperado else 'No'}")

Mensaje: abc
Hash SHA-256: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Hash esperado: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
¿El hash coincide?: Sí


In [25]:
from utils import run_sha256_multiple_times

In [26]:
# Ejemplo de uso
mensaje = ["abc"]*30
csv_file = "sha256_data.csv"
run_sha256_multiple_times(messages=mensaje, csv_file=csv_file,tipo="implementacion",func=sha256)
print(f"Resultados de la implementacion guardados en {csv_file}")

mensaje = ["abc"]*30
csv_file = "sha256_data.csv"
run_sha256_multiple_times(messages=mensaje, csv_file=csv_file,tipo="libreria",func=calculate_sha256_hash)
print(f"Resultados de la libreria guardados en {csv_file}")

Resultados de la implementacion guardados en sha256_data.csv
Resultados de la libreria guardados en sha256_data.csv


In [27]:
import pandas as pd 

In [28]:
df = pd.read_csv('sha256_data.csv')

In [29]:
df.head()

,tipo,mensaje,hash,tiempo_ejecucion (s)
0,implementacion,abc,ba7816bf8f01cfea414140de5dae2223b00361a396177a...,0.000445
1,implementacion,abc,ba7816bf8f01cfea414140de5dae2223b00361a396177a...,0.000441
2,implementacion,abc,ba7816bf8f01cfea414140de5dae2223b00361a396177a...,0.000424
3,implementacion,abc,ba7816bf8f01cfea414140de5dae2223b00361a396177a...,0.000402
4,implementacion,abc,ba7816bf8f01cfea414140de5dae2223b00361a396177a...,0.000446


In [31]:
conclusion = df.groupby("tipo").agg(tiempo_ejecucion=('tiempo_ejecucion (s)','mean'))
conclusion.head()

,tiempo_ejecucion
tipo,
implementacion,0.000478
libreria,0.000017


In [35]:
# Cálculo del porcentaje y tipo más rápido
tiempo_libreria = conclusion.loc["libreria", "tiempo_ejecucion"]
tiempo_implementacion = conclusion.loc["implementacion", "tiempo_ejecucion"]

if tiempo_libreria < tiempo_implementacion:
    porcentaje = (1 - tiempo_libreria / tiempo_implementacion) * 100
    tipo = "libreria"
else:
    porcentaje = (1 - tiempo_implementacion / tiempo_libreria) * 100
    tipo = "implementacion"

# Impresión del resultado
print(f"La {tipo} es {porcentaje:.2f}% más rápida.")


La libreria es 96.42% más rápida.


# Explicacion del Codigo

### Explicación del Código: Implementación del Algoritmo SHA-256

El código implementa una versión simplificada del algoritmo **SHA-256**, que es un algoritmo de hash criptográfico ampliamente utilizado. A continuación, se desglosan sus componentes principales:

---

#### 1. **Constantes SHA-256**
```python
K = [...]
```
Estas son constantes específicas del estándar SHA-256 (definidas en la RFC 6234). Se utilizan durante las rondas de compresión y derivan de las raíces cúbicas de números primos.

---

#### 2. **Funciones Auxiliares**
```python
def rotr(x, n, w=32):
    return (x >> n) | ((x << (w - n)) & (2**w - 1))
```
- `rotr` (rotate right): Realiza una rotación hacia la derecha en un número de 32 bits (`x`).
- La rotación se utiliza en varias funciones internas de SHA-256 para mezclar bits.

---

#### 3. **Extensión de Bloques y Compresión**
##### 3.1 **Extensión de Mensajes**
```python
W = list(struct.unpack(">16L", chunk)) + [0] * 48
for t in range(16, 64):
    s0 = rotr(W[t - 15], 7) ^ rotr(W[t - 15], 18) ^ (W[t - 15] >> 3)
    s1 = rotr(W[t - 2], 17) ^ rotr(W[t - 2], 19) ^ (W[t - 2] >> 10)
    W[t] = (W[t - 16] + s0 + W[t - 7] + s1) & 0xFFFFFFFF
```
- **W**: Se extiende un bloque de 512 bits (64 bytes) a 64 palabras de 32 bits.
- Se utilizan operaciones de rotación y desplazamiento para crear una expansión no lineal de los datos.

##### 3.2 **Rondas de Compresión**
```python
for t in range(64):
    S1 = rotr(e, 6) ^ rotr(e, 11) ^ rotr(e, 25)
    ch = (e & f) ^ (~e & g)
    temp1 = (h + S1 + ch + K[t] + W[t]) & 0xFFFFFFFF
    S0 = rotr(a, 2) ^ rotr(a, 13) ^ rotr(a, 22)
    maj = (a & b) ^ (a & c) ^ (b & c)
    temp2 = (S0 + maj) & 0xFFFFFFFF
    # Actualización de variables
    h = g
    g = f
    f = e
    e = (d + temp1) & 0xFFFFFFFF
    d = c
    c = b
    b = a
    a = (temp1 + temp2) & 0xFFFFFFFF
```
- Se realizan **64 rondas** de compresión:
  - `S1`, `S0`: Operaciones de mezcla de bits mediante rotaciones.
  - `ch` (choose): Selección basada en los valores de los bits de `e`, `f`, `g`.
  - `maj` (majority): Selección basada en la mayoría de los bits de `a`, `b`, `c`.
  - `temp1` y `temp2`: Actualizan las variables hash intermedias.
- **Objetivo**: Mezclar los valores iniciales con los datos del mensaje para garantizar una salida no reversible.

##### 3.3 **Actualización del Hash**
```python
return [
    (H[0] + a) & 0xFFFFFFFF,
    ...
    (H[7] + h) & 0xFFFFFFFF,
]
```
Después de las rondas, se combinan los valores intermedios con el hash inicial.

---

#### 4. **Preprocesamiento**
```python
def sha256_padding(message):
    message_bytes = bytearray(message, 'ascii')
    original_length = len(message_bytes) * 8
    message_bytes.append(0x80)
    while len(message_bytes) % 64 != 56:
        message_bytes.append(0)
    message_bytes += struct.pack(">Q", original_length)
    return message_bytes
```
El mensaje se procesa para cumplir los requisitos de entrada de SHA-256:
1. Se añade un bit `1` al final del mensaje.
2. Se rellenan ceros hasta que la longitud sea un múltiplo de 512 bits menos 64 bits.
3. Se añaden los 64 bits finales que representan la longitud original del mensaje.

---

#### 5. **Algoritmo Principal**
```python
def sha256(message):
    H = [
        0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
        0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19,
    ]
    padded_message = sha256_padding(message)
    for i in range(0, len(padded_message), 64):
        chunk = padded_message[i:i + 64]
        H = sha256_compress(chunk, H)
    return ''.join(f'{x:08x}' for x in H)
```
1. **Inicialización**:
   - Se definen valores hash iniciales (`H`), derivados de las raíces cuadradas de números primos.
2. **Preprocesamiento**:
   - El mensaje se ajusta para cumplir con las especificaciones de SHA-256.
3. **Procesamiento por Bloques**:
   - El mensaje se divide en bloques de 512 bits.
   - Cada bloque se comprime con `sha256_compress`.
4. **Generación del Hash**:
   - Se convierte el valor final de `H` en una cadena hexadecimal de 64 caracteres (256 bits).

---

### Resumen
- **SHA-256** procesa mensajes en bloques de 512 bits, mezclándolos con valores iniciales fijos.
- Utiliza operaciones no lineales (rotaciones, desplazamientos) y combinaciones booleanas para garantizar un hash único.
- Este código implementa el algoritmo paso a paso siguiendo las especificaciones del estándar.

COMPARAR EL QE HICE CON EL DE LA BIBLIOTECA